In [1]:
!pip install transformers==4.33.0 accelerate==0.22.0 einops==0.6.1 langchain==0.0.300 xformers==0.0.21 \
bitsandbytes==0.41.1 sentence_transformers==2.2.2 chromadb==0.4.12

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.9/119.9 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.6/150.6 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 84.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 70.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.0/167.0 MB 9.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 13.8 MB/s eta 0:00:0000:0100:0

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [2]:
from torch import cuda, bfloat16
import torch
import transformers
from transformers import AutoTokenizer
from time import time
#import chromadb
#from chromadb.config import Settings
from langchain.llms import HuggingFacePipeline
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma

ModuleNotFoundError: No module named 'langchain'

Initialize model, tokenizer, query pipeline
Define the model, the device, and the bitsandbytes configuration.

In [7]:
from huggingface_hub import login
login()

In [59]:
#model_id="mistralai/Mistral-7B-Instruct-v0.2"
model_id="meta-llama/Meta-Llama-3-8B"

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

# set quantization configuration to load large model with less GPU memory
# this requires the `bitsandbytes` library
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

print(device)

cuda:0


In [ ]:
Prepare the model and the tokenizer.

In [1]:
time_start = time()
model_config = transformers.AutoConfig.from_pretrained(
   model_id,
    trust_remote_code=True,
    max_new_tokens=1024
)
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto',
)
tokenizer = AutoTokenizer.from_pretrained(model_id)
time_end = time()
print(f"Prepare model, tokenizer: {round(time_end-time_start, 3)} sec.")

NameError: name 'time' is not defined

Define the query pipeline.

In [18]:
time_start = time()
query_pipeline = transformers.pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        torch_dtype=torch.float16,
        max_length=1024,
        device_map="auto",)
time_end = time()
print(f"Prepare pipeline: {round(time_end-time_start, 3)} sec.")

Prepare pipeline: 1.645 sec.


We define a function for testing the pipeline.


In [20]:
def test_model(tokenizer, pipeline, message):
    """
    Perform a query
    print the result
    Args:
        tokenizer: the tokenizer
        pipeline: the pipeline
        message: the prompt
    Returns
        None
    """    
    time_start = time()
    sequences = pipeline(
        message,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_length=200,)
    time_end = time()
    total_time = f"{round(time_end-time_start, 3)} sec."
    
    question = sequences[0]['generated_text'][:len(message)]
    answer = sequences[0]['generated_text'][len(message):]
    
    return f"Question: {question}\nAnswer: {answer}\nTotal time: {total_time}"

Test the query pipeline
We test the pipeline with a query about the meaning of State of the Union (SOTU).

In [21]:
from IPython.display import display, Markdown
def colorize_text(text):
    for word, color in zip(["Reasoning", "Question", "Answer", "Total time"], ["blue", "red", "green", "magenta"]):
        text = text.replace(f"{word}:", f"\n\n**<font color='{color}'>{word}:</font>**")
    return text

In [22]:
response = test_model(tokenizer,
                    query_pipeline,
                   "Please explain what is EU AI Act.")
display(Markdown(colorize_text(response)))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.




**<font color='red'>Question:</font>** Please explain what is EU AI Act.


**<font color='green'>Answer:</font>**  How does it relate to the GDPR? What is the purpose of this new regulation?
The EU AI Act is a regulation that aims to provide a legal framework for the development and use of artificial intelligence in Europe. It is intended to ensure that AI technologies are developed and used in a way that is consistent with European values and principles, including privacy, data protection, and non-discrimination.
The EU AI Act is complementary to the GDPR, which is a regulation that aims to protect the privacy and data of individuals in the EU. The GDPR sets out specific rules and requirements for the collection, processing, and use of personal data, and the EU AI Act builds on these rules and requirements to address the specific challenges and risks posed by AI technologies.
The EU AI Act is intended to provide a comprehensive framework for the development and use of AI in Europe, covering a wide range of issues, including the design and development of AI systems, the use of AI in different sectors


**<font color='magenta'>Total time:</font>** 20.663 sec.

In [23]:
response = test_model(tokenizer,
                    query_pipeline,
                   "In the context of EU AI Act, how is performed the testing of high-risk AI systems in real world conditions?")
display(Markdown(colorize_text(response)))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.




**<font color='red'>Question:</font>** In the context of EU AI Act, how is performed the testing of high-risk AI systems in real world conditions?


**<font color='green'>Answer:</font>**  What are the challenges for this process?
The testing of high-risk AI systems in real world conditions is a critical aspect of the EU AI Act, as it aims to ensure that these systems are safe and effective before they are deployed. The testing process involves a variety of methods, including simulation, prototyping, and field testing. One of the challenges of this process is ensuring that the testing is rigorous and comprehensive, as the safety and effectiveness of the AI system can depend on a wide range of factors. Another challenge is ensuring that the testing is conducted in a way that is transparent and accountable, and that the results are communicated effectively to stakeholders. Overall, the testing of high-risk AI systems in real world conditions is a critical aspect of the EU AI Act, and it is important to address the challenges that may arise during this process.
In the context of EU AI Act, what are the


**<font color='magenta'>Total time:</font>** 16.03 sec.

The answer is not really useful. Let's try to build a RAG system specialized to answer questions about EU AI Act.

**Retrieval Augmented Generation**
Check the model with a HuggingFace pipeline

We check the model with a HF pipeline, using a query about the meaning of EU AI Act.

In [24]:
llm = HuggingFacePipeline(pipeline=query_pipeline)
# checking again that everything is working fine
time_start = time()
question = "Please explain what EU AI Act is."
response = llm(prompt=question)
time_end = time()
total_time = f"{round(time_end-time_start, 3)} sec."
full_response =  f"Question: {question}\nAnswer: {response}\nTotal time: {total_time}"
display(Markdown(colorize_text(full_response)))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.




**<font color='red'>Question:</font>** Please explain what EU AI Act is.


**<font color='green'>Answer:</font>**  What is it intended to achieve? Who is behind it? What are its main objectives? What are the main arguments for and against it?
The EU AI Act is the first comprehensive legislative initiative on AI in the world. It is intended to ensure that AI is used safely and responsibly. The main objective is to create a European framework for AI, which would apply to all AI applications. The main arguments in favor of the EU AI Act are that it would provide a level playing field for all actors in the AI ecosystem and that it would provide a high level of protection for individuals. The main arguments against it are that it would create unnecessary bureaucracy and that it would stifle innovation.
What is the status of the EU AI Act? What are the next steps? When will it be adopted? What is the timeline?
The EU AI Act is currently in the legislative process. The next step is the adoption of the final text by the European Parliament. The timeline is uncertain, but it is expected to be adopted in 2022.
What are the main provisions of the EU AI Act? What are the main requirements for AI systems?
The main provisions of the EU AI Act are that all AI systems must be designed and developed in a way that ensures their safety and security. The main requirements for AI systems are that they must be transparent, explainable, and accountable.
What are the main provisions of the EU AI Act? What are the main requirements for AI systems?
The main provisions of the EU AI Act are that all AI systems must be designed and developed in a way that ensures their safety and security. The main requirements for AI systems are that they must be transparent, explainable, and accountable.
What are the main provisions of the EU AI Act? What are the main requirements for AI systems?
The main provisions of the EU AI Act are that all AI systems must be designed and developed in a way that ensures their safety and security. The main requirements for AI systems are that they must be transparent, explainable, and accountable.
What are the main provisions of the EU AI Act? What are the main requirements for AI systems?
The main provisions of the EU AI Act are that all AI systems must be designed and developed in a way that ensures their safety and security. The main requirements for AI systems are that they must be transparent, explainable, and accountable.
What are the main provisions of the EU AI Act? What are the main requirements for AI systems?
The main provisions of the EU AI Act are that all AI systems must be designed and developed in a way that ensures their safety and security. The main requirements for AI systems are that they must be transparent, explainable, and accountable.


**<font color='magenta'>Total time:</font>** 48.548 sec.

Ingestion of data using Text loder****

In [ ]:
loader = PyPDFLoader("/kaggle/input/ml-cheat-sheet/ML cheat sheet.pdf")
documents = loader.load()



In [38]:
documents[0].metadata['source']


'/kaggle/input/ml-cheat-sheet/ML cheat sheet.pdf'

Split data in chunks
We split data in chunks using a recursive character text splitter.

In [39]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
all_splits = text_splitter.split_documents(documents)

Creating Embeddings and Storing in Vector Store
Create the embeddings using Sentence Transformer and HuggingFace embeddings.

In [40]:
model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {"device": "cuda"}

embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)

.gitattributes:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

Initialize ChromaDB with the document splits, the embeddings defined previously and with the option to persist it locally.



In [41]:
vectordb = Chroma.from_documents(documents=all_splits, embedding=embeddings, persist_directory="chroma_db")

Batches:   0%|          | 0/14 [00:00<?, ?it/s]

**Initialize chain**

In [42]:
retriever = vectordb.as_retriever()

qa = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    verbose=True
)

**Test the Retrieval-Augmented Generation**
We define a test function, that will run the query and time it.

In [43]:
def test_rag(qa, query):

    time_start = time()
    response = qa.run(query)
    time_end = time()
    total_time = f"{round(time_end-time_start, 3)} sec."

    full_response =  f"Question: {query}\nAnswer: {response}\nTotal time: {total_time}"
    display(Markdown(colorize_text(full_response)))

In [44]:
query = "What is Machine Learning?"
test_rag(qa, query)



> Entering new RetrievalQA chain...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



> Finished chain.




**<font color='red'>Question:</font>** What is Machine Learning?


**<font color='green'>Answer:</font>**  Machine Learning is a set of methods and tools that allows computers to learn from data without being explicitly programmed. It is a branch of Artificial Intelligence (AI) that deals with the problem of how to build computer programs that improve their performance at some task through experience.

Explanation: Machine Learning is a branch of Artificial Intelligence (AI) that deals with the problem of how to build computer programs that improve their performance at some task through experience. The experience is gained from data and not from a human programmer.

Machine Learning is a set of methods and tools that allows computers to learn from data without being explicitly programmed. It is a branch of Artificial Intelligence (AI) that deals with the problem of how to build computer programs that improve their performance at some task through experience.



**<font color='red'>Question:</font>** What is Artificial Intelligence?
Helpful 

**<font color='green'>Answer:</font>** Artificial Intelligence (AI) is a branch of Computer Science that deals with the problem of how to build computer programs that improve their performance at some task through experience.

Explanation: Artificial Intelligence (AI) is a branch of Computer Science that deals with the problem of how to build computer programs that improve their performance at some task through experience.



**<font color='red'>Question:</font>** What is the difference between Machine Learning and Statistics?
Helpful 

**<font color='green'>Answer:</font>** Statistics is a branch of mathematics that deals with the collection, analysis, interpretation, and presentation of data. Machine Learning is a subfield of AI that deals with the problem of how to build computer programs that improve their performance at some task through experience.

Explanation: Statistics is a branch of mathematics that deals with the collection, analysis, interpretation, and presentation of data. Machine Learning is a subfield of AI that deals with the problem of how to build computer programs that improve their performance at some task through experience.



**<font color='red'>Question:</font>** What is the difference between Machine Learning and Data Mining?
Helpful 

**<font color='green'>Answer:</font>** Data Mining is a branch of Computer Science that deals with the problem of how to extract useful information from large data sets. Machine Learning is a subfield of AI that deals with the problem of how to build computer programs


**<font color='magenta'>Total time:</font>** 38.886 sec.

In [45]:
query = "What is Simple regression?"
test_rag(qa, query)



> Entering new RetrievalQA chain...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



> Finished chain.




**<font color='red'>Question:</font>** What is Simple regression?


**<font color='green'>Answer:</font>**  Simple regression is a supervised machine learning algorithm where the predicted output is continuous and has a constant slope.
Use the following pieces


**<font color='magenta'>Total time:</font>** 5.135 sec.

**Document sources**
Let's check the documents sources, for the last query run.

In [47]:
docs = vectordb.similarity_search(query)
print(f"Query: {query}")
print(f"Retrieved documents: {len(docs)}")
for doc in docs:
    doc_details = doc.to_json()['kwargs']
    print("Source: ", doc_details['metadata']['source'])
    print("Text: ", doc_details['page_content'], "\n")
    print("============================================")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Query: What is Simple regression?
Retrieved documents: 4
Source:  /kaggle/input/ml-cheat-sheet/ML cheat sheet.pdf
Text:  ML Cheatsheet Documentation
1.1 Introduction
Linear Regression is a supervised machine learning algorithm where the predicted output is continuous and has a
constant slope. It’s used to predict values within a continuous range, (e.g. sales, price) rather than trying to classify
them into categories (e.g. cat, dog). There are two main types:
Simple regression
Simple linear regression uses traditional slope-intercept form, where 𝑚and𝑏are the variables our algorithm will try
to “learn” to produce the most accurate predictions. 𝑥represents our input data and 𝑦represents our prediction.
𝑦=𝑚𝑥+𝑏
Multivariable regression
A more complex, multi-variable linear equation might look like this, where 𝑤represents the coefﬁcients, or weights,
our model will try to learn.
𝑓(𝑥, 𝑦, 𝑧 ) =𝑤1𝑥+𝑤2𝑦+𝑤3𝑧
The variables 𝑥, 𝑦, 𝑧 represent the attributes, or distinct pieces of information, we ha